In [ ]:
# Customer Lifetime Value Modeling with PyMC: Hierarchical BG/NBD, Gamma-Gamma, and Cohort Decay

# 📦 Dependencies
!pip install pymc pymc-marketing pandas matplotlib seaborn numpy

# 📚 Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az

# 🎲 Step 1: Simulate Sample Transaction Data
np.random.seed(42)
n_customers = 500
channels = ['Organic', 'Paid', 'Referral']

customer_df = pd.DataFrame({
    'customer_id': [f'C{i}' for i in range(n_customers)],
    'channel': np.random.choice(channels, size=n_customers, p=[0.4, 0.4, 0.2]),
})

customer_df['acquisition_date'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 30, n_customers), unit='D')

transactions = []
for _, row in customer_df.iterrows():
    cust_id = row['customer_id']
    channel = row['channel']
    start_date = row['acquisition_date']
    num_txn = np.random.poisson(3 if channel == 'Organic' else 2 if channel == 'Referral' else 1.5)
    churn_prob = 0.2 if channel == 'Organic' else 0.35 if channel == 'Referral' else 0.5
    order_value_mean = 60 if channel == 'Organic' else 90 if channel == 'Paid' else 70

    date = start_date
    for _ in range(num_txn):
        if np.random.rand() < churn_prob:
            break
        amount = np.random.gamma(order_value_mean / 10, 10)
        transactions.append([cust_id, date, amount, channel])
        date += pd.to_timedelta(np.random.randint(7, 60), unit='D')
        if date > pd.to_datetime('2023-06-30'):
            break

transactions_df = pd.DataFrame(transactions, columns=['customer_id', 'purchase_date', 'revenue', 'channel'])
transactions_df.to_csv("sample_transactions.csv", index=False)

# 📊 Step 2: Prepare Summary for Hierarchical Modeling
transactions_df = pd.read_csv("sample_transactions.csv")
transactions_df['purchase_date'] = pd.to_datetime(transactions_df['purchase_date'])

# Assign cohort index
grouped_channels, cohort_codes = pd.factorize(transactions_df['channel'])
transactions_df['channel_code'] = grouped_channels

from pymc_marketing.utils import rfm_summary

summary = rfm_summary(
    transactions_df,
    customer_id_col="customer_id",
    datetime_col="purchase_date",
    monetary_value_col="revenue",
    observation_period_end=pd.to_datetime("2023-07-01")
)

customer_channels = transactions_df.drop_duplicates(subset='customer_id')[['customer_id', 'channel', 'channel_code']]
summary = summary.merge(customer_channels, left_index=True, right_on='customer_id')

# 🧠 Step 3: Hierarchical BG/NBD + Gamma-Gamma Model with Custom Priors
with pm.Model() as hierarchical_ltv_model:
    n_cohorts = len(cohort_codes)
    cohort_idx = summary['channel_code'].values

    mu_r = pm.Gamma("mu_r", alpha=2, beta=0.5)
    sigma_r = pm.HalfCauchy("sigma_r", beta=1)
    mu_alpha = pm.Gamma("mu_alpha", alpha=2, beta=0.5)
    sigma_alpha = pm.HalfCauchy("sigma_alpha", beta=1)
    mu_a = pm.Gamma("mu_a", alpha=2, beta=0.5)
    sigma_a = pm.HalfCauchy("sigma_a", beta=1)
    mu_b = pm.Gamma("mu_b", alpha=2, beta=0.5)
    sigma_b = pm.HalfCauchy("sigma_b", beta=1)

    r = pm.LogNormal("r", mu=mu_r, sigma=sigma_r, shape=n_cohorts)
    alpha = pm.LogNormal("alpha", mu=mu_alpha, sigma=sigma_alpha, shape=n_cohorts)
    a = pm.LogNormal("a", mu=mu_a, sigma=sigma_a, shape=n_cohorts)
    b = pm.LogNormal("b", mu=mu_b, sigma=sigma_b, shape=n_cohorts)

    freq = summary['frequency'].values
    rec = summary['recency'].values
    T = summary['T'].values

    ll_bgnbd = (
        pm.math.gammaln(r[cohort_idx] + freq)
        - pm.math.gammaln(r[cohort_idx])
        + r[cohort_idx] * pm.math.log(alpha[cohort_idx])
        + freq * pm.math.log(rec + alpha[cohort_idx])
        - (r[cohort_idx] + freq) * pm.math.log(T + alpha[cohort_idx])
    )
    pm.Potential("bgnbd_likelihood", ll_bgnbd.sum())

    mv = summary['monetary_value'].values
    p = pm.Gamma("p", alpha=2, beta=0.5)
    q = pm.Gamma("q", alpha=2, beta=0.5)
    v = pm.Gamma("v", alpha=2, beta=0.5)

    pm.Gamma("monetary_obs", alpha=p, beta=v / (freq + q), observed=mv)

    trace = pm.sample(1000, tune=1000, target_accept=0.9, return_inferencedata=True)

# 📈 Posterior Diagnostics
az.plot_trace(trace)
plt.suptitle("Hierarchical Model Posterior Trace", fontsize=14)
plt.show()

az.plot_posterior(trace)
plt.suptitle("Posterior Distributions by Cohort", fontsize=14)
plt.show()

# 📋 Inference Summary
inference_summary = az.summary(trace, round_to=2)
print("\nCustom Inference Report:\n")
print(inference_summary[['mean', 'hdi_3%', 'hdi_97%', 'ess_bulk', 'r_hat']])

# 📉 Empirical Cohort Decay Curves
transactions_df['week'] = transactions_df['purchase_date'].dt.to_period('W').apply(lambda r: r.start_time)
cohort_retention = transactions_df.groupby(['channel', 'week'])['customer_id'].nunique().reset_index()
cohort_retention['week_num'] = cohort_retention.groupby('channel')['week'].rank().astype(int)

plt.figure(figsize=(12, 6))
for channel in channels:
    temp = cohort_retention[cohort_retention['channel'] == channel]
    base = temp['customer_id'].iloc[0]
    plt.plot(temp['week_num'], temp['customer_id'] / base, label=channel)

plt.title('Empirical Cohort Decay Curves by Channel')
plt.xlabel('Week since acquisition')
plt.ylabel('% of active customers')
plt.legend()
plt.grid(True)
plt.show()
